In [10]:
import pandas as pd
import os

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

from datetime import date
from datetime import timedelta

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from io import BytesIO

import boto3, os, io

from datetime import datetime

from twilio.rest import Client
import time


In [11]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="Config.env")

twilio_acc = os.getenv('TWILIO_ACC')
twilio_secret = os.getenv('TWILIO_SECRET')
twilio_phone = os.getenv('TWILIO_PHONE')
WEATHER_API = os.getenv('WEATHER_API')

my_key= os.getenv('AWS_KEY') 
my_secret= os.getenv('AWS_SECRET')

query = 'Poza Rica'


In [12]:
def GetResponse(city,dateQuery,WEATHER_API):
    url = "http://api.weatherapi.com/v1/history.json?key=" + WEATHER_API + "&q=" + city + "&dt=" + dateQuery
    return requests.get(url).json()

In [13]:
def get_forecast(response,i):
    general = response['forecast']['forecastday'][0]['hour'][i]
    fecha= general['time'].split()[0]
    hour = general['time'].split()[1].split(":")[0]
    condition = general["condition"]['text']
    temp_C = general["temp_c"]
    will_rain = general["will_it_rain"]
    chance_of_rain = general['chance_of_rain']
    
    return fecha,hour,condition,temp_C,will_rain,chance_of_rain
def getTimeForecastData(response):
    data = []
    for x in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour="green"):
        data.append(get_forecast(response,x))
    return data

In [36]:
bucket_name = 'weather-innova-proto'


def UploadToS3(query,dateQuery,bucket_name,my_key,my_secret,data):
    filename = query +"-"+dateQuery
    filename = filename.replace(" ","") + '.csv'
    s3 = boto3.resource('s3',aws_access_key_id=my_key,aws_secret_access_key=my_secret) 
    csv_buf = BytesIO()
    data.to_csv(csv_buf, header=True, index=False)
    csv_buf.seek(0)
    try:
        s3.Object(bucket_name, filename).put(Body=csv_buf)
        return "succesful upload"
    except(e):
        return e
    
    

In [37]:
def UploadDateRange(city,dateStart,numberDays,S3bucket,S3Key,S3secret,weatherApiKey):
    for x in range(numberDays):
        dateToQuery = (dateStart - timedelta(days = x)).strftime("%Y-%m-%d")
        response = GetResponse(query,dateToQuery,weatherApiKey)
        data = getTimeForecastData(response)
        col = ["fecha","hour","condition","temp_C","will_rain","chance_of_rain"]
        df = pd.DataFrame(data,columns=col)
        UploadToS3(city,dateToQuery,S3bucket,S3Key,S3secret,df)
    return "Complete Succefully"
    

In [38]:
UploadDateRange(query,date.today(),15,bucket_name,my_key,my_secret,WEATHER_API)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


'Complete Succefully'